In [16]:
try:
    from bs4 import BeautifulSoup
except:
    !pip install beautifulsoup4
    from bs4 import BeautifulSoup

try:
    import pycountry
except:
    !pip install pycountry
    import pycountry

import requests
import json
from urllib.parse import urljoin

In [17]:
country_list_url = "https://fci.be/en/statistics/"

response = requests.get(country_list_url)
soup = BeautifulSoup(response.content, 'html.parser')

In [18]:
def retrieve_info(country_url):
    response = requests.get(country_url)
    soup = BeautifulSoup(response.content, 'html.parser')
    table = soup.find("div", class_="center content").find("table")
    rows = table.find_all("tr")

    dogs_count_by_year = {}
    years_row = rows[0]

    for row in rows[1:]:
        cells = row.find_all("td")
        if len(cells) > 0 and "Approximate TOTAL number of DOGS" in cells[0].text:
            for cell_index in range(1, len(cells)):
                year = int(years_row.find_all("td")[cell_index].text.strip())
                dogs_count_str = cells[cell_index].text.strip().replace(",", "")

                dogs_count_by_year[year] = int(dogs_count_str) if dogs_count_str else None

    return dogs_count_by_year

result = []

country_lists = soup.find_all("ul", class_="listepays")
for country_list in country_lists:
    for country in country_list.find_all("li"):
        country_name = country.find("a").text.strip()
        country_url = urljoin(country_list_url, country.find("a")["href"])

        iso_alpha2 = country_url.split("?")[1].split("iso=")[1].split("&")[0]
        country_info = pycountry.countries.get(alpha_2=iso_alpha2)
        iso_alpha3 = country_info.alpha_3 if country_info else ""

        print(country_name, iso_alpha2, iso_alpha3)

        info = retrieve_info(country_url)

        for year, dogs_count in info.items():
            result.append({
                "country_name": country_name.title(),
                "iso_alpha2": iso_alpha2,
                "iso_alpha3": iso_alpha3,
                "year": year,
                "dogs_count": dogs_count
            })

with open("dog-stats.json", "w") as f:
    json.dump(result, f, indent=2)


ARGENTINA AR ARG
BOLIVIA BO BOL
BRAZIL BR BRA
CHILE CL CHL
COLOMBIA CO COL
COSTA RICA CR CRI
CUBA CU CUB
DOMINICAN REPUBLIC DO DOM
ECUADOR EC ECU
EL SALVADOR SV SLV
GUATEMALA GT GTM
HONDURAS HN HND
MEXICO MX MEX
NICARAGUA NI NIC
PANAMA PA PAN
PARAGUAY PY PRY
PERU PE PER
PUERTO RICO PR PRI
URUGUAY UY URY
VENEZUELA (BOLIVARIAN REPUBLIC OF) VE VEN
ALBANIA AL ALB
ARMENIA AM ARM
AUSTRIA AT AUT
AZERBAIJAN AZ AZE
BELARUS BY BLR
BELGIUM BE BEL
BOSNIA AND HERZEGOVINA BA BIH
BULGARIA BG BGR
CROATIA HR HRV
CYPRUS CY CYP
CZECH REPUBLIC CZ CZE
DENMARK DK DNK
ESTONIA EE EST
FINLAND FI FIN
FRANCE FR FRA
GEORGIA GE GEO
GERMANY DE DEU
GIBRALTAR GI GIB
GREECE GR GRC
HUNGARY HU HUN
ICELAND IS ISL
IRELAND IE IRL
ISRAEL IL ISR
ITALY IT ITA
KAZAKHSTAN KZ KAZ
KIRGHIZISTAN KG KGZ
KOSOVO KV 
LATVIA LV LVA
LEBANON LB LBN
LITHUANIA LT LTU
LUXEMBOURG LU LUX
MALTA MT MLT
MOLDAVIA (REPUBLIC OF) MD MDA
MONACO MC MCO
MONTENEGRO ME MNE
NORTH MACEDONIA MK MKD
NORWAY NO NOR
POLAND PL POL
PORTUGAL PT PRT
ROMANIA RO ROU
R